# getting started
If you have mamba or conda installed you can use the cell below to create an environment that can be used to run this jupyter notebook in jupyter or vscode
```python
mamba env create --file './nostr.yml'
mamba activate nostr
```
or
```python
conda env create --file './nostr.yml'
conda activate nostr
```

# python-nostr examples
this notebook serves as a set of interactive examples to highlight how nostr and python-nostr work. the cell below will make sure this library is in your python path

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path('../').resolve()))
print(sys.path)

['/Users/ryanarmstrong/Documents/01_Programming/GitHub/python-nostr/example_nbs', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python310.zip', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python3.10', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python3.10/lib-dynload', '', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python3.10/site-packages', '/Users/ryanarmstrong/Documents/01_Programming/GitHub/python-nostr']



## key management
python-nostr has basic key management capabilities to generate new private keys or load existing keys.
The base key class is instantiated with raw_bytes - if you are loading an existing hex key you can use `Key.raw_bytes_from_hex()`

The `PublicKey` and `PrivateKey` classes inherit these capabilities. The bech32 representation of the private and public key (i.e. npub... and nsec...) are defined in each class respectively.

**change log from original repo:**
 - Added base Key class to manage shared methods
 - Added hex to key method
 - Changed `raw_secret` to `raw_bytes` in `PrivateKey` to keep classes consistent
    - this change helps the code, but does it add some risk to users?

In [2]:
from pathlib import Path
import secrets
from nostr.key import Key, PrivateKey, PublicKey

base_key = Key(secrets.token_bytes(32))
print(f'this is the raw bytes of a key: {base_key.raw_bytes}\n'
      f'this is the hex of the same key: {base_key.hex()}')

assert base_key.hex() == Key.raw_bytes_from_hex(base_key.hex()).hex()
assert issubclass(PrivateKey, Key) and issubclass(PublicKey, Key)

try:
      base_key.bech32()
except ValueError as e:
      print(e)

this is the raw bytes of a key: b'X\xcbX\x18\xc0\x08\xce\xd8\x83\xc5\xe1\xb6 \xac\xc7q\x93\xd0N\xb2\x96N"\x90I\xce\xc1I\x1c[\x19C'
this is the hex of the same key: 58cb5818c008ced883c5e1b620acc77193d04eb2964e229049cec1491c5b1943
bech32 prefix not defined for type <class 'nostr.key.Key'>


Below we use the `PrivateKey` class to make a private key or load it from a file on disk. We can also confirm that the private and public key representations start with the appropriate characters (npub and nsec)

In [3]:
key_file = Path('./private_key.txt') # saving a private key for testing

if not key_file.exists():
    private_key = PrivateKey()
    public_key = private_key.public_key
    print(f"Generated new private key")
    print(f"Public key: {public_key.bech32()}")
    with open(key_file, 'w') as f:
        f.write(private_key.raw_bytes.hex())
else:
    with open(key_file, 'r') as f:
        hex = f.read()
        # private_key = PrivateKey(bytes.fromhex(hex))
        private_key = PrivateKey.raw_bytes_from_hex(hex)
    public_key = private_key.public_key
    print(f'Loaded private key from {key_file}')
    print(f'Public key: {public_key.bech32()}')

assert private_key.bech32()[:4]=='nsec' and public_key.bech32()[:4]=='npub'

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


## relay manangement
Next let's instantiate a `RelayManager` and add a couple relay urls. We are going to set one to not write and see what happens

In [4]:
from nostr.relay_manager import RelayManager, Relay

relay_manager = RelayManager()
relay_manager.add_relay('wss://nostr-2.zebedee.cloud')
relay_manager.add_relay('wss://nostr.zebedee.cloud')
relay_manager.add_relay('wss://nostr-relay.lnmarkets.com', write=False)

the relay manager keeps all of the `Relay` objects in a dict keyed by the url. Let's look at some of the properties of the relays we added before we actually do anything

**changelog from original repo:**
 - added `__repr__` of json object for relay

In [5]:
for url, relay in relay_manager.relays.items():
    print(relay, '\n\n')
    assert type(relay) == Relay and url == relay.url


{
  "url": "wss://nostr-2.zebedee.cloud",
  "policy": {
    "read": true,
    "write": true
  },
  "subscriptions": []
} 


{
  "url": "wss://nostr.zebedee.cloud",
  "policy": {
    "read": true,
    "write": true
  },
  "subscriptions": []
} 


{
  "url": "wss://nostr-relay.lnmarkets.com",
  "policy": {
    "read": true,
    "write": false
  },
  "subscriptions": []
} 




Now we can connect - in this case we will connect and test that all relays have a websocket (just a naive test that they are not None when open and are None when closed).

**changelog from original repo:**
 - added a connection context for easier operations
    - is this actually helpful in context of nostr?


In [6]:
import ssl
from nostr.relay_manager import Connection

with relay_manager.connection({"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
    for relay in relay_manager.relays.values():
        assert relay.ws.sock is not None

for relay in relay_manager.relays.values():
    assert relay.ws.sock is None

## writing to the relays

In [7]:
import json 
import ssl
import time
from nostr.event import Event
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType
from nostr.key import PrivateKey


event = Event(private_key.public_key.hex(), "Hello Nostr - testing from python-nostr")
event.sign(private_key.hex())

message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])

with relay_manager.connection({"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
    time.sleep(2)
    relay_manager.publish_message(message)
    time.sleep(1) # allow the messages to send

## requesting from the relays
we can use python-nostr to make requests to send to the relays. First we will try to request the event id we just published.

> **_NOTE:_** when an event is processed by python noster it's ID is stored in `RelayManager.message_pool._unique_events`. This ID won't get processed a second time. If you are just exploring in a notebook like this you will need to clear this attribue to process the event a second time.

In [8]:
import json
import ssl
import time
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType


filters = Filters([Filter(ids=[event.id])])
subscription_id = 'single_event'
request = [ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())
message = json.dumps(request)

# checking what we made above
print(f'the request is: {request}')
print(f'the message to the relay is: {message}')

relay_manager.add_subscription(subscription_id, filters)
print('The following subscriptions have been added to relays:')
for url, relay in relay_manager.relays.items():
    print(f'{url}:', relay.subscriptions.get(subscription_id)
                                  .to_json_object())

the request is: ['REQ', 'single_event', {'ids': ['235bfa0ecbc1f8126634ada51397e15d25f1972e23396982333dc7fbbd203f2c']}]
the message to the relay is: ["REQ", "single_event", {"ids": ["235bfa0ecbc1f8126634ada51397e15d25f1972e23396982333dc7fbbd203f2c"]}]
The following subscriptions have been added to relays:
wss://nostr-2.zebedee.cloud: {'id': 'single_event', 'filters': [{'ids': ['235bfa0ecbc1f8126634ada51397e15d25f1972e23396982333dc7fbbd203f2c']}]}
wss://nostr.zebedee.cloud: {'id': 'single_event', 'filters': [{'ids': ['235bfa0ecbc1f8126634ada51397e15d25f1972e23396982333dc7fbbd203f2c']}]}
wss://nostr-relay.lnmarkets.com: {'id': 'single_event', 'filters': [{'ids': ['235bfa0ecbc1f8126634ada51397e15d25f1972e23396982333dc7fbbd203f2c']}]}


In [9]:
import warnings

with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
  previous_event_ids = len(relay_manager.message_pool._unique_events)
  print(f'{previous_event_ids} events already received\n')
  if event.id in relay_manager.message_pool._unique_events:
    warnings.warn(f'event {event.id} has already been processed and won\'t display below. '
                   'Please reinitiate the message pool to receive it again')
                   
  time.sleep(1.25) # allow the connections to open
  relay_manager.publish_message(message)
  time.sleep(1) # allow the messages to send

  while relay_manager.message_pool.has_events():
    event_msg = relay_manager.message_pool.get_event()
    print(event_msg.event.to_json_object())
  while relay_manager.message_pool.has_notices():
    notice_msg = relay_manager.message_pool.get_notice()
    print(notice_msg.event.to_json_object())
  while relay_manager.message_pool.has_eose_notices():
    eose_notice_msg = relay_manager.message_pool.get_eose_notice()
    print(eose_notice_msg.subscription_id)

0 events already received

{'id': '235bfa0ecbc1f8126634ada51397e15d25f1972e23396982333dc7fbbd203f2c', 'pubkey': '2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3', 'created_at': 1671997215, 'kind': 1, 'tags': [], 'content': 'Hello Nostr - testing from python-nostr', 'sig': '473fce768882aabffaebbaf4cd894bc4cc55e5c3d962ab3e7fad4de96bd8afcd97c1faacb95f6662a02b6f0bd32b291955b755052e63b7748f31816171696790'}
single_event


Now let's try to request the last 10 events from a user

**TODO:** add other event types and examples below

In [10]:
import json
import ssl
import time
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType


filters = Filters([Filter(authors=['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'],
                          kinds=[EventKind.TEXT_NOTE],
                          limit=10
                          )])
subscription_id = 'last10_armstrys'
request = [ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())
message = json.dumps(request)

# checking what we made above
print(f'the request is: {request}')
print(f'the message to the relay is: {message}')

relay_manager.add_subscription(subscription_id, filters)
print('The following subscriptions have been added to relays:')
for url, relay in relay_manager.relays.items():
    print(f'{url}:', relay.subscriptions.get(subscription_id)
                                  .to_json_object())

the request is: ['REQ', 'last10_armstrys', {'kinds': [<EventKind.TEXT_NOTE: 1>], 'authors': ['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'], 'limit': 10}]
the message to the relay is: ["REQ", "last10_armstrys", {"kinds": [1], "authors": ["c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720"], "limit": 10}]
The following subscriptions have been added to relays:
wss://nostr-2.zebedee.cloud: {'id': 'last10_armstrys', 'filters': [{'kinds': [<EventKind.TEXT_NOTE: 1>], 'authors': ['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'], 'limit': 10}]}
wss://nostr.zebedee.cloud: {'id': 'last10_armstrys', 'filters': [{'kinds': [<EventKind.TEXT_NOTE: 1>], 'authors': ['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'], 'limit': 10}]}
wss://nostr-relay.lnmarkets.com: {'id': 'last10_armstrys', 'filters': [{'kinds': [<EventKind.TEXT_NOTE: 1>], 'authors': ['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'], 'limit': 10}]

In [11]:
import warnings

with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
  previous_event_ids = len(relay_manager.message_pool._unique_events)
  print(f'{previous_event_ids} events already received\n')

  time.sleep(1.25) # allow the connections to open
  relay_manager.publish_message(message)
  time.sleep(1) # allow the messages to send
  
  while relay_manager.message_pool.has_events():
    event_msg = relay_manager.message_pool.get_event()
    print(event_msg.event.to_json_object())
  while relay_manager.message_pool.has_notices():
    notice_msg = relay_manager.message_pool.get_notice()
    print(notice_msg.event.to_json_object())
  while relay_manager.message_pool.has_eose_notices():
    eose_notice_msg = relay_manager.message_pool.get_eose_notice()
    print(eose_notice_msg.subscription_id)

1 events already received

{'id': '80eb69eb0bb3fb723793b0a3a64ee3e9d4c82858fde57e24e512c17ae065c1f2', 'pubkey': 'c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720', 'created_at': 1671603398, 'kind': 1, 'tags': [['e', 'a04a2834cd3dde0240a4bc005f2f0d28f7ce1407aa7432637652518561ee5f8f'], ['e', 'acc16aa9e37542b90bb3679108d646befca0232dcebcacfa1fa3dbdd152adc85'], ['p', 'ba67bf89761da5ed59966077c4d514e94bb11195ce81d252572e3bac14e6803b']], 'content': 'yes, astral.ninja is an example of a client that allows you to change. I think it maybe be coming to Damus soon along with a few other features, but unsure', 'sig': '72fbe7df8593c28ac1c06cb2a4d85e502fcff34a9e02b5722ac2435f588b70fb19e7c43949c2741bf19b1b4f46da8db2dbddc4463b2da1b57b2fc41f3c3893f1'}
{'id': '22c6b307cc8aa2d434a292258109b83e3e96befe04b9a342fde7ca5be8154683', 'pubkey': 'c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720', 'created_at': 1671565919, 'kind': 1, 'tags': [['e', '307c775e05d43368dc5ac3a7deeecbfb6